# RUN: {Name for run}

```
Lead     : {your name / github id} (ii = initials)
Issue    : {github issue # if applicable}
Version  : {git rev-parse HEAD}
start    : YYYY MM DD
complete : YYYY MM DD
files    : {~/serratus/notebook/YYMMDD_ii/}           # Small repo files / plots
s3_files : {s3://serratus-public/notebook/YYMMDD_ii/} # Bucket containing large (>500 kb) data files
output   : {s3://serratus-public/out/YYMMDD_ii/}      # Output data files (bam/flagstat/etc...)
```

### Objectives
- {Primary objective of this run}
- {Secondary objective of this run}


## Serratus Initialization

Local system initialization procedures for `serratus`

### Initialize local workspace

In [ ]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS
git rev-parse HEAD # commit version

In [ ]:
# Create local run directory
WORK="$SERRATUS/notebook/200423_ab"
mkdir -p $WORK; cd $WORK

In [ ]:
# SRA RunInfo Table for run
aws s3 cp s3://serratus-public/sra/testing_SraRunInfo.csv ./
RUNINFO="$WORK/testing_SraRunInfo.csv"
cat $RUNINFO

### Packer / AMI Initialization (optional)
Does not need to be ran each time if you have access to the AMI already.

Current Build: `us-east-1: ami-046baafb2ee438b69`

In [ ]:
cd $SERRATUS/packer
packer build docker-ami.json

### Build Serratus containers (optional)
Serratus containers are available on the `serratusbio` dockerhub. If you wish to deploy your own containers, you will have to build them from the `serratus` repository and upload them to your own dockerhub account.

This can be done with the `build.sh` script

In [ ]:
cd $SERRATUS

# If you want to upload containers to your repository
# include this.
export DOCKERHUB_USER='serratusbio' # optional
sudo docker login # optional

# Build all containers and upload them docker hub repo
# (if available)
./build.sh

### Terraform Initialization
The Global Variables for Terraform file must be modified to initialize for your system.

File: `$SERRATUS/terarform/main/terraform.tfvars`

This step must be done manually in a text editor currently.

In [ ]:
## Change these parameters in $SERRATUS/terarform/main/terraform.tfvars

# Your public IP followed by "/32"
LOCALIP="75.155.242.67/32" #dev_cidrs
# Your AWS key name
KEYNAME="serratus"         #key_name
# Dockerhub account containing serratus containers
DOCKERHUB_USER='serratusbio'    #dockerhub_account (optional)

In [ ]:
# Initialize terraform
cd $SERRATUS/terraform/main
terraform init

In [ ]:
# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

## Running Serratus 
Upload the run data, scale-out the cluster, monitor performance.


### Run Monitors & Upload table

Open SSH tunnels to monitor node then open monitors in browser

- [Scheduler Table](localhost:8000/jobs/)
- [Cluster Monitor: Grafana](http://localhost:3000/?orgId=1)
- [Cluster Monitor: Prometheus](http://localhost:9090)


#### Empty Scheduler Table (localhost:8000/jobs/)
![Empty Table Load Screen](200423_ab/empty_scheduler.png)

#### Ready Scheduler Table (localhost:8000/jobs/)
![Empty Table Load Screen](200423_ab/ready_scheduler.png)

In [ ]:
cd $SERRATUS/terraform/main

# Open SSH tunnels to the monitor
./create_tunnels.sh

In [ ]:
# Load SRA Run Info into scheduler (READY)
curl -s -X POST -T $RUNINFO localhost:8000/jobs/add_sra_run_info/

### Scale up the cluster
This will set-up 10 download, 10 align and 2 merge nodes to process data


In [ ]:
./dl_set_capacity.sh 10
./align_set_capacity.sh 10
./merge_set_capacity.sh 2

You can track the progress of accessions in the scheduler:

![Running Scheduler](200423_ab/running_scheduler2.png)

And monitor the performance of the cluster in the monitor:

![Running Monitor](200423_ab/running_monitor.png)

In [ ]:
# When all downloading/splitting is done,
# scale-in the downloaders
./dl_set_capacity.sh 0

In [ ]:
# When all alignment is done,
# scale-in the aligners
./align_set_capacity.sh 0

In [ ]:
# When all merging is done,
# scale in the mergers
./merge_set_capacity.sh 0

In [ ]:
# Dump the Scheduler SQLITE table to a local file
curl localhost:8000/db > \
  $SERRATUS/notebook/200423_ab/schedDump.sqlite

## Shutting down procedures

Closing up shop.


### Save output of runs


In [ ]:
# Output files are in two folders:
# Bam and Bai files
aws s3 ls s3://tf-serratus-work-<id>/out/bam/
# Flagstat and RefCount files
aws s3 ls s3://tf-serratus-work-<id>/out/flagstat/


In [ ]:
# Copy output to a permenant bucket
# TODO: automatically transfer final outputs
# to the permenant bucket
aws s3 sync \
  s3://tf-serratus-work-<id>/out \
  s3://serratus-public/out/YYMMDD_ii/


## Destroy Cluster

Close out all resources with terraform (will take a few minutes).


In [ ]:
terraform destroy -auto-approve
# WARNING this will also delete the standard output bucket/data
# Save data prior to destroy

# Run Notes
Notes on this run for future reference.

## Errors


## Performance


## Output
